In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import psycopg2
from geoalchemy2 import Geometry, WKTElement
from sqlalchemy import *
from shapely.geometry import MultiPolygon
from zipfile import ZipFile
import requests 
import sys

In [2]:
import yaml

with open('../../config/postgres.yaml') as f:
    engine_configs = yaml.load(f, Loader=yaml.FullLoader)
    
try:
    engine = create_engine('postgresql://{username}:{password}@{host}:{port}/{dbname}'.format(**engine_configs))
except Exception as e:
    print("Uh oh, can't connect. Invalid dbname, user or password?")
    print(e)

In [3]:
def process_geometry_SQL_insert(gdf):
    gdf['geom'] = gdf['geometry'].apply(lambda x: WKTElement((MultiPolygon([x]) if x.geom_type == 'Polygon' else x).wkt, srid=4326))
    gdf = gdf.drop('geometry', 1)
    return gdf

# Often when reading in a ShapeFile from Basemap, you'll get: "ValueError: readshapefile can only handle 2D shape types"
# A trick can be to convert your geometry in your GeoPandas Dataframe and restoring the new flattened 2D geometry
# series back into a shapefile and try again.

# edit from http://stackoverflow.com/questions/33417764/basemap-readshapefile-valueerror  

from shapely.geometry import Polygon, MultiPolygon, shape, Point
def convert_3D_2D(geometry):
    '''
    Takes a GeoSeries of 3D Multi/Polygons (has_z) and returns a list of 2D Multi/Polygons
    '''
    new_geo = []
    for p in geometry:
        if p.has_z:
            if p.geom_type == 'Polygon':
                lines = [xy[:2] for xy in list(p.exterior.coords)]
                new_p = Polygon(lines)
                new_geo.append(new_p)
            elif p.geom_type == 'MultiPolygon':
                new_multi_p = []
                for ap in p:
                    lines = [xy[:2] for xy in list(ap.exterior.coords)]
                    new_p = Polygon(lines)
                    new_multi_p.append(new_p)
                new_geo.append(MultiPolygon(new_multi_p))
    return new_geo


In [4]:
CITY='chicago'

### Neighborhoods

In [5]:
sql = """INSERT INTO spatial_groups (city, core_geom, core_id, lower_ids, spatial_name, approx_geom)
SELECT a.city, a.core_geom, a.core_id, array_agg(a.core_id), 'core', ST_multi(a.core_geom)
FROM spatial_groups a
where a.city='{city}' and a.spatial_name = 'ego'
GROUP BY a.core_id, a.core_geom, a.city;
""".format(city=CITY, tempname=CITY.lower())

result = engine.execute(text(sql))

## Land use

In [6]:
land_gdf = gpd.read_file('zip://../../data/chicago/land_use/land_use.zip', dtype={'LANDUSE': str})
land_gdf = land_gdf.to_crs({'init': 'epsg:4326'}) 
land_gdf.head()

/home/denadai/.local/share/virtualenvs/crime-environment-u9RldRjF/lib/python3.6/site-packages/pyproj/crs/crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,LANDUSE,OS_MGMT,FAC_NAME,PLATTED,MODIFIER,Shape_Leng,Shape_Area,geometry
0,1111,None,None,None,None,608.931265,20878.200769,"POLYGON ((-88.19304 41.20262, -88.19342 41.202..."
1,1111,None,None,None,None,787.202461,38719.346410,"POLYGON ((-88.18844 41.20272, -88.18879 41.202..."
2,1111,None,None,None,None,931.208086,45822.457370,"POLYGON ((-88.15646 41.20339, -88.15646 41.203..."
3,1111,None,None,None,None,740.975876,33953.759347,"POLYGON ((-88.11770 41.20415, -88.11845 41.204..."
4,1111,None,None,None,None,1396.015199,109212.531687,"POLYGON ((-88.12903 41.20399, -88.13071 41.203..."


In [7]:
land_gdf['landuse'] = 'none'

land_gdf.loc[(land_gdf['LANDUSE'].str[:2].isin({'11'})) | (land_gdf['LANDUSE'].isin({'1216'})), 'landuse'] = 'residential'
land_gdf.loc[(land_gdf['LANDUSE'].str[:2].isin({'12', '13', '14', '15', '20'})) & (~land_gdf['LANDUSE'].isin({'1510', '1511', '1512', '1520', '1550', '1561', '1565'})), 'landuse'] = 'commercial'
 
land_gdf.loc[land_gdf['LANDUSE'].str[:1].isin({'3'}), 'landuse'] = 'recreational'
land_gdf.loc[land_gdf['LANDUSE'].str[:1].isin({'4'}), 'landuse'] = 'vacant'
land_gdf.head()

,LANDUSE,OS_MGMT,FAC_NAME,PLATTED,MODIFIER,Shape_Leng,Shape_Area,geometry,landuse
0,1111,None,None,None,None,608.931265,20878.200769,"POLYGON ((-88.19304 41.20262, -88.19342 41.202...",residential
1,1111,None,None,None,None,787.202461,38719.346410,"POLYGON ((-88.18844 41.20272, -88.18879 41.202...",residential
2,1111,None,None,None,None,931.208086,45822.457370,"POLYGON ((-88.15646 41.20339, -88.15646 41.203...",residential
3,1111,None,None,None,None,740.975876,33953.759347,"POLYGON ((-88.11770 41.20415, -88.11845 41.204...",residential
4,1111,None,None,None,None,1396.015199,109212.531687,"POLYGON ((-88.12903 41.20399, -88.13071 41.203...",residential


## Net area

In [8]:
land_gdf_unique = land_gdf.copy()

land_gdf_unique.loc[:, 'x'] = land_gdf_unique.geometry.centroid.x
land_gdf_unique.loc[:, 'y'] = land_gdf_unique.geometry.centroid.y
land_gdf_unique = land_gdf_unique.drop_duplicates(subset=['x', 'y'])[['geometry', 'landuse']]

In [9]:
ins_gdf = process_geometry_SQL_insert(land_gdf_unique)
ins_gdf.to_sql('temptable_unique_{}'.format(CITY.lower()), engine, if_exists='replace', index=False, dtype={'geom': Geometry('MultiPolygon', srid=4326)})

In [10]:
sql = """
UPDATE temptable_unique_{tempname} p SET geom=ST_Multi(ST_buffer(p.geom, 0.0)) 
WHERE NOT ST_Isvalid(p.geom);
""".format(city=CITY, tempname=CITY.lower())

result = engine.execute(text(sql))

In [11]:
## This deletes the blocks that are related to streets
sql = """
DELETE FROM block b
WHERE city='{city}' and NOT EXISTS (select * from temptable_unique_{tempname} t where st_intersects(t.geom, b.geom) and t.landuse <> 'none');
""".format(city=CITY, tempname=CITY.lower())

result = engine.execute(text(sql))

In [12]:
sql = """
DELETE 
FROM temptable_unique_{tempname} t
USING unused_areas u 
WHERE u.city = '{city}' AND ST_Intersects(u.geom, t.geom) AND (NOT ST_Touches(u.geom, t.geom)) 
AND (ST_Contains(u.geom, t.geom) OR ST_AREA(ST_Intersection(t.geom, u.geom))/ST_Area(t.geom) > 0.5);
""".format(city=CITY, tempname=CITY.lower())

result = engine.execute(text(sql))

In [13]:
sql = """
INSERT INTO spatial_groups_net_area (sp_id, city, spatial_name, used_area) 
SELECT sp_id, city, spatial_name, SUM(ST_Area(ST_Intersection(s.approx_geom, t.geom)::geography))/1000000.
FROM temptable_unique_{tempname} t
INNER JOIN spatial_groups s ON ST_Intersects(s.approx_geom, t.geom) AND NOT ST_Touches(s.approx_geom, t.geom)
WHERE s.city = '{city}' AND s.spatial_name='core' 
GROUP BY sp_id, city, spatial_name;
""".format(city=CITY, tempname=CITY.lower())

result = engine.execute(text(sql))

## Refresh materialized views

In [14]:
sql = """
REFRESH MATERIALIZED VIEW spatial_groups_unused_areas;
"""

result = engine.execute(text(sql))

In [ ]:
2